In [1]:
# Install Java, Spark, and Findspark
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 252 kB in 2s (157 kB/s)
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-16 19:45:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.44MB/s    in 0.6s    

2020-08-16 19:45:07 (1.44 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://databootcamp-uci.s3-us-west-1.amazonaws.com/amazon_reviews_us_Sports_v1_00.tsv"
spark.sparkContext.addFile(url)
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

In [5]:
from pyspark.sql.types import IntegerType, StringType, BooleanType

In [6]:
clean_data_df = user_data_df.withColumn("customer_id", user_data_df["customer_id"].cast(StringType()))

In [7]:
clean_data_df = clean_data_df.withColumn("product_parent", clean_data_df["product_parent"].cast(StringType()))

In [8]:
clean_data_df = clean_data_df.withColumn("star_rating", clean_data_df["star_rating"].cast(IntegerType()))

In [9]:
clean_data_df = clean_data_df.withColumn("vine", clean_data_df["vine"].cast(BooleanType()))

In [10]:
clean_data_df = clean_data_df.withColumn("verified_purchase", clean_data_df["verified_purchase"].cast(BooleanType()))

In [11]:
clean_data_df = clean_data_df.dropna()

In [12]:
clean_data_df = clean_data_df.dropDuplicates()

In [13]:
clean_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: boolean (nullable = true)
 |-- verified_purchase: boolean (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [14]:
no_vine_df = clean_data_df.filter(clean_data_df["vine"]==False)

In [15]:
vine_df = clean_data_df.filter(clean_data_df["vine"]==True)

In [17]:
no_vine_helpful = clean_data_df.filter((clean_data_df["vine"]==False) & (clean_data_df["helpful_votes"]>0))

In [18]:
vine_helpful = clean_data_df.filter((clean_data_df["vine"]==True) & (clean_data_df["helpful_votes"]>0))

In [19]:
no_vine_helpful.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes| vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+-----+-----------------+--------------------+--------------------+-------------------+
|         US|   52467947| ROVT8XIK2DDO0|B00HWJGMGI|     457555201|Moultrie M-1100i ...|          Sports|          3|            3|          3|false|             true|but my cell phone...|I bought 5 of the...|2015-08-31 00:00:00|
|         US|     247695|R380AFN78YW0X4|B00O45DJLY|     577863295|Maxpedition Kodia.

In [20]:
no_vine_helpful.select("star_rating").describe().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|           1518546|
|   mean| 3.991325913077378|
| stddev|1.4000271744629806|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [21]:
vine_helpful.select("star_rating").describe().show()

+-------+------------------+
|summary|       star_rating|
+-------+------------------+
|  count|              4442|
|   mean| 4.138226024313372|
| stddev|0.9834396874214884|
|    min|                 1|
|    max|                 5|
+-------+------------------+



In [22]:
no_vine_rank =  [int(row.star_rating) for row in no_vine_helpful.select("star_rating").collect()]

In [23]:
vine_rank =  [int(row.star_rating) for row in vine_helpful.select("star_rating").collect()]

In [24]:
import scipy.stats as st

2 Sample T-test  : to veriy whether average of star_rating from vine and the other is statistically different.

In [25]:
st.ttest_ind(vine_rank, no_vine_rank, equal_var=False)

Ttest_indResult(statistic=9.926134694923544, pvalue=5.515745741864647e-23)

Conclusion : 2 Sample T-test shows the average of star_rating from vine and the other is statistically different.

ANOVA Test

In [26]:
st.f_oneway(vine_rank, no_vine_rank)

F_onewayResult(statistic=48.83411704338422, pvalue=2.7866552337033935e-12)

Conclusion: ANOVA Test shows averages of star_rating values from vine and the other are significantly different.
It mean Vine Group gives higher star rating than others.